# **Librerias usadas**

In [68]:
import pandas as pd
import numpy as np
from meteostat import Stations, Daily
from datetime import datetime
import csv
import os
import requests
import re
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import folium
import random

# **Web scraping**

## **Sección 1: Configuración e Infraestructura (Scraping)**

Este bloque se encarga de obtener la lista maestra de estaciones.

In [123]:
import requests
import re
import pandas as pd

# --- FUNCIÓN DE EXTRACCIÓN (WEB SCRAPING) ---
def extraer_desde_html():
    """
    Escanea la web del SENAMHI para extraer coordenadas ocultas de 969 estaciones.
    """
    print("🌐 Iniciando Scraping de Infraestructura SENAMHI...")
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        # 1. Petición a la web
        response = requests.get(url, headers=headers, timeout=10)
        contenido = response.text
        
        # 2. Extracción con Regex (Patrón: nombre, latitud, longitud)
        regex = r'"nom"\s*:\s*"(.*?)".*?"lat"\s*:\s*(-?\d+\.?\d*).*?"lon"\s*:\s*(-?\d+\.?\d*)'
        matches = re.findall(regex, contenido)
        
        if matches:
            print(f"✅ ¡ÉXITO! Se detectaron {len(matches)} estaciones activas.")
            # Guardamos en CSV para uso del sistema
            df = pd.DataFrame(matches, columns=["ESTACION", "LATITUD", "LONGITUD"])
            df["LATITUD"] = pd.to_numeric(df["LATITUD"])
            df["LONGITUD"] = pd.to_numeric(df["LONGITUD"])
            df.to_csv("SENAMHI_ESTACIONES_FINAL.csv", index=False)
            return df
        else:
            print("⚠️ No se encontraron patrones. Revisar estructura web.")
            return pd.DataFrame()

    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return pd.DataFrame()
        
if __name__ == "__main__":
    extraer_desde_html()

🌐 Iniciando Scraping de Infraestructura SENAMHI...
✅ ¡ÉXITO! Se detectaron 968 estaciones activas.


## **Sección 2: Lógica Geoespacial (Buscador de Vecinos)**

Este bloque recibe al avión y encuentra la estación física. Se ha fusionado la búsqueda con la validación.

In [131]:
# --- BUSCADOR GEODÉSICO Y AUDITORÍA ---
def encontrar_y_validar_estacion(lat_obj, lon_obj, df_estaciones):
    """
    Encuentra la estación más cercana y audita su confiabilidad por distancia.
    """
    # 1. Cálculo Vectorial de Distancia (Euclidiana simple para velocidad)
    df_estaciones['distancia'] = np.sqrt(
        (df_estaciones['LATITUD'] - lat_obj)**2 + 
        (df_estaciones['LONGITUD'] - lon_obj)**2
    ) * 111  # Factor aprox de grados a km

    # 2. Seleccionar la ganadora
    mejor_estacion = df_estaciones.sort_values('distancia').iloc[0]
    dist_km = round(mejor_estacion['distancia'], 2)
    nombre = mejor_estacion['ESTACION']

    # 3. Semáforo de Confianza (Auditoría)
    if dist_km <= 5.0:
        confianza = "🟢 ALTA (Ideal)"
    elif dist_km <= 20.0:
        confianza = "🟡 MEDIA (Aceptable)"
    else:
        confianza = "🔴 BAJA (Descartar)"

    return {
        "ESTACION": nombre,
        "LAT": mejor_estacion['LATITUD'],
        "LON": mejor_estacion['LONGITUD'],
        "DIST_KM": dist_km,
        "CONFIANZA": confianza}

## **Sección 3: Inteligencia Climática (Meteostat + Feature Engineering)**

Este bloque descarga la historia real y calcula el riesgo operativo.

In [136]:
# --- MOTOR DE INTELIGENCIA CLIMÁTICA ---
def analizar_riesgo_climatico(lat, lon, inicio, fin):
    """
    Descarga datos históricos y calcula indicadores de riesgo operativo.
    """
    # 1. Conexión a Meteostat (API Histórica)
    try:
        estaciones = Stations().nearby(lat, lon)
        estacion_id = estaciones.fetch(1).index[0]
        data = Daily(estacion_id, inicio, fin).fetch()
    except:
        return pd.DataFrame() # Retorna vacío si falla

    if data.empty: return pd.DataFrame()

    # 2. Feature Engineering (Cálculo de Indicadores)
    df = data.copy()
    
    # Rellenar vacíos técnicos
    df['wspd'] = df['wspd'].fillna(df['wspd'].median())
    df['pres'] = df['pres'].fillna(1013.0) # Presión estándar
    df['prcp'] = df['prcp'].fillna(0)

    # Definir Umbrales de Riesgo
    umbral_viento = df['wspd'].quantile(0.90) # Top 10% más fuerte
    umbral_nieve_temp = 2.0 # Grados C

    # 3. Clasificador de Severidad (Lógica de Negocio)
    condiciones = []
    for index, row in df.iterrows():
        score = 0
        if row['wspd'] > umbral_viento: score += 1
        if row['prcp'] > 5.0: score += 1
        
        # Detector Forense de Nieve
        if row['prcp'] > 0 and row['tmin'] <= umbral_nieve_temp:
            etiqueta = "❄️ SEVERA (Nieve)"
        elif score >= 2:
            etiqueta = "🔴 SEVERA (Tormenta)"
        elif score == 1:
            etiqueta = "🟡 MODERADA"
        else:
            etiqueta = "🟢 NORMAL"
        condiciones.append(etiqueta)

    df['CONDICION_OPERATIVA'] = condiciones
    return df[['tmin', 'tmax', 'prcp', 'wspd', 'pres', 'CONDICION_OPERATIVA']]

## **¿Qué tiempo hace en esas estaciones y qué tan peligroso es para volar?**

## **Sección 4: Ejecución Maestra**

Este es el único bloque que necesitas ejecutar al final para demostrar que todo funciona junto.

In [140]:
# --- ORQUESTADOR PRINCIPAL (MERGE UNION) ---
if __name__ == "__main__":
    print("🚀 INICIANDO PIPELINE DE VALIDACIÓN AÉREA...\n")

    # 1. INPUT: Coordenadas de un vuelo (Ej. Aproximación a Lima)
    lat_vuelo, lon_vuelo = -12.02, -77.11 
    print(f"📍 Aeronave detectada en: {lat_vuelo}, {lon_vuelo}")

    # 2. ETAPA A: Infraestructura (Scraping o Carga)
    try:
        df_base = pd.read_csv("SENAMHI_ESTACIONES_FINAL.csv")
    except:
        df_base = extraer_desde_html() # Si no existe, scrapea

    # 3. ETAPA B: Correlación Espacial
    resultado_geo = encontrar_y_validar_estacion(lat_vuelo, lon_vuelo, df_base)
    print(f"\n📡 Estación SENAMHI Vinculada: {resultado_geo['ESTACION']}")
    print(f"📏 Distancia al objetivo: {resultado_geo['DIST_KM']} km")
    print(f"🚦 Auditoría de Confianza: {resultado_geo['CONFIANZA']}")

    # 4. ETAPA C: Inteligencia Operativa
    print("\n⚡ Analizando Riesgo Histórico (2024)...")
    df_riesgo = analizar_riesgo_climatico(
        resultado_geo['LAT'], 
        resultado_geo['LON'], 
        datetime(2024, 1, 1), 
        datetime(2024, 12, 31)
    )

    # 5. RESULTADOS
    print("\n📊 REPORTE DE OPERACIONES (Últimos 5 días):")
    print(df_riesgo.tail(5).to_string())
    
    # 6. GENERAR MAPA DE EVIDENCIA
    print("\n🎨 Generando mapa de auditoría...")
    m = folium.Map(location=[lat_vuelo, lon_vuelo], zoom_start=13)
    folium.Marker([lat_vuelo, lon_vuelo], icon=folium.Icon(color="red", icon="plane", prefix="fa"), popup="AVIÓN").add_to(m)
    folium.Marker([resultado_geo['LAT'], resultado_geo['LON']], icon=folium.Icon(color="green", icon="cloud", prefix="fa"), popup="SENAMHI").add_to(m)
    folium.PolyLine([[lat_vuelo, lon_vuelo], [resultado_geo['LAT'], resultado_geo['LON']]], color="blue", dash_array='5, 10').add_to(m)
    folium.Circle([resultado_geo['LAT'], resultado_geo['LON']], radius=5000, color="green", fill=True, fill_opacity=0.1).add_to(m)
    m.save("MAPA_FINAL.html")
    print("✅ Mapa guardado como 'MAPA_FINAL.html'")

🚀 INICIANDO PIPELINE DE VALIDACIÓN AÉREA...

📍 Aeronave detectada en: -12.02, -77.11

📡 Estación SENAMHI Vinculada: SAN MARTIN DE PORRES
📏 Distancia al objetivo: 3.09 km
🚦 Auditoría de Confianza: 🟢 ALTA (Ideal)

⚡ Analizando Riesgo Histórico (2024)...

📊 REPORTE DE OPERACIONES (Últimos 5 días):
            tmin  tmax  prcp  wspd    pres CONDICION_OPERATIVA
time                                                          
2024-12-27  21.0  26.0   0.0  18.6  1013.9            🟢 NORMAL
2024-12-28  19.0  23.0   0.0  18.9  1013.0            🟢 NORMAL
2024-12-29  19.0  23.0   0.0  19.3  1012.9            🟢 NORMAL
2024-12-30  20.0  23.2   0.0  16.8  1012.7            🟢 NORMAL
2024-12-31  19.0  24.0   0.0  13.1  1012.6            🟢 NORMAL

🎨 Generando mapa de auditoría...
✅ Mapa guardado como 'MAPA_FINAL.html'


# **Motor de búsqueda y enriquecimiento operativo**

## **Análisis exploratorio de datos**

Objetivo:
- Mostrar el DataFrame.head()
- Explicar qué variables se extrajeron
- info()
- describe()

In [84]:
# ===============================
# 1. OBTENER ESTACIONES EN PERÚ
# ===============================
def obtener_estaciones_peru():
    estaciones = Stations()
    estaciones = estaciones.region('PE')
    df_estaciones = estaciones.fetch()
    return df_estaciones


# ===============================
# 2. OBTENER DATOS CLIMÁTICOS
# ===============================
def obtener_datos_climaticos(lat, lon, inicio, fin):
    estaciones = Stations().nearby(lat, lon)
    estacion = estaciones.fetch(1)

    if estacion.empty:
        return pd.DataFrame()

    data = Daily(estacion.index[0], inicio, fin)
    df = data.fetch()
    return df


# ===============================
# 3. EJECUCIÓN PRINCIPAL
# ===============================
if __name__ == "__main__":

    # --- Paso 1: Estaciones ---
    print(" Obteniendo estaciones en Perú...")
    estaciones_pe = obtener_estaciones_peru()

    print("\n Vista previa de estaciones:")
    print(estaciones_pe.head())

    # --- Paso 2: Datos climáticos (ejemplo Lima) ---
    lat_lima = -12.0464
    lon_lima = -77.0428

    inicio = datetime(2024, 1, 1)
    fin = datetime(2024, 12, 31)

    print("\n Descargando datos climáticos para Lima...")
    df_clima = obtener_datos_climaticos(lat_lima, lon_lima, inicio, fin)

    # --- Paso 3: Guardar CSV ---
    nombre_csv = "clima_lima_2024.csv"
    df_clima.to_csv(nombre_csv)
    print(f"\n Archivo CSV generado: {nombre_csv}")

    # --- Paso 4: Mostrar DataFrame ---
    print("\n Vista previa del DataFrame climático:")
    print(df_clima.head())

    print("\n Información del DataFrame:")
    print(df_clima.info())

    print("\n Estadísticas descriptivas:")
    print(df_clima.describe())


 Obteniendo estaciones en Perú...

 Vista previa de estaciones:
              name country region    wmo  icao  latitude  longitude  \
id                                                                    
84370       Tumbes      PE     TU  84370  SPME   -3.5500     -80.40   
84377      Iquitos      PE     LO  84377  SPQT   -3.7500     -73.25   
84390       Talara      PE     PI  84390  SPYL   -4.5667     -81.25   
84401        Piura      PE     PI  84401  SPUR   -5.1833     -80.60   
84405  Huancabamba      PE     PI  84405  SPAB   -5.2333     -79.45   

       elevation      timezone hourly_start hourly_end daily_start  daily_end  \
id                                                                              
84370       25.0  America/Lima   1974-03-15 2025-12-15  1974-03-15 2025-05-27   
84377      125.0  America/Lima   1973-01-01 2025-12-15  1973-01-01 2025-06-24   
84390       85.0  America/Lima   1933-02-09 2025-12-14  1942-11-01 2025-08-24   
84401       49.0  America/Lima   

Conclusión:
- El análisis exploratorio permitió validar la estructura y calidad del conjunto de datos, identificando variables relevantes y descartando aquellas con información incompleta, asegurando así una base adecuada para análisis posteriores.

## **Caracterización Climática y Generación de Indicadores Meteorológicos**

Objetivo: 
- Transformar los datos climáticos crudos del SENAMHI en indicadores meteorológicos operativos, listos para ser usados más adelante por el pipeline aéreo.

### **Selección de variables meteorológicas relevantes**

| Variable | Uso operativo                    |
| -------- | -------------------------------- |
| `wspd`   | Impacto en despegue y aterrizaje |
| `prcp`   | Riesgo de pista mojada           |
| `pres`   | Estabilidad atmosférica          |
| `tmax`   | Densidad del aire                |
| `tmin`   | Condiciones nocturnas            |

- Se seleccionaron variables meteorológicas con impacto directo en la operación aérea, descartando aquellas sin registros completos.

### **Limpieza y tratamiento de valores faltantes**

Objetivo:
- Garantizar la calidad y consistencia del conjunto de datos climático antes de su transformación en indicadores operativos.

### **Construcción de indicadores climáticos**

Objetivo:
- Transformar las variables meteorológicas continuas en indicadores que permitan detectar condiciones climáticas potencialmente adversas.
- A partir de las variables climáticas seleccionadas se construyeron indicadores binarios que permiten identificar eventos meteorológicos relevantes para la operación aérea.

Indicadores definidos:
- Viento fuerte
- Precipitación intensa
- Presión atmosférica baja

### **Clasificación de condiciones meteorológicas**

Objetivo:
- Etiquetar cada observación temporal según el nivel de severidad climática, facilitando su integración futura con datos operativos.
- Finalmente, se clasificaron las condiciones meteorológicas en categorías de severidad climática, permitiendo una interpretación directa del impacto potencial del clima.

Categorías:
- Normal
- Viento adverso
- Precipitación intensa
- Condición severa

## **Código-SENAMHI**

### **Extracción de datos**

In [101]:
def obtener_datos_climaticos(lat, lon, inicio, fin):
    est = Stations().nearby(lat, lon).fetch(1)
    if est.empty:
        return pd.DataFrame()
    return Daily(est.index[0], inicio, fin).fetch()

# --- EJECUCIÓN ---
df = obtener_datos_climaticos(
    -12.0464, -77.0428,
    datetime(2024, 1, 1),
    datetime(2024, 12, 31))

print(df)

            tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt    pres  tsun
time                                                                    
2024-01-01  23.4  22.0  25.2   1.0  <NA>  <NA>  19.1  <NA>  1014.0  <NA>
2024-01-02  23.8  22.0  26.0   0.0  <NA>  <NA>  20.9  <NA>  1013.4  <NA>
2024-01-03  23.5  22.0  25.3   0.0  <NA>  <NA>  21.5  <NA>  1013.5  <NA>
2024-01-04  23.6  21.8  26.0   0.0  <NA>  <NA>  16.4  <NA>  1014.1  <NA>
2024-01-05  23.9  22.0  26.0   0.0  <NA>  <NA>  24.7  <NA>  1013.2  <NA>
...          ...   ...   ...   ...   ...   ...   ...   ...     ...   ...
2024-12-27  22.5  21.0  26.0   0.0  <NA>  <NA>  18.6  <NA>  1013.9  <NA>
2024-12-28  21.1  19.0  23.0   0.0  <NA>  <NA>  18.9  <NA>  1013.0  <NA>
2024-12-29  20.9  19.0  23.0   0.0  <NA>  <NA>  19.3  <NA>  1012.9  <NA>
2024-12-30  21.4  20.0  23.2   0.0  <NA>  <NA>  16.8  <NA>  1012.7  <NA>
2024-12-31  21.2  19.0  24.0   0.0  <NA>  <NA>  13.1  <NA>  1012.6  <NA>

[366 rows x 10 columns]


### **Limpieza de datos**

In [103]:
def limpiar_datos_climaticos(df):
    columnas_utiles = ["tavg", "tmin", "tmax", "prcp", "wspd", "pres"]
    df_limpio = df[columnas_utiles].copy()
    return df_limpio

### **Construcción de indicadores**

In [105]:
def construir_indicadores(df):
    df_ind = df.copy()

    # Rellenar NA SOLO para indicadores
    df_ind["prcp"] = df_ind["prcp"].fillna(0)
    df_ind["wspd"] = df_ind["wspd"].fillna(df_ind["wspd"].median())
    df_ind["pres"] = df_ind["pres"].fillna(df_ind["pres"].median())

    # Umbrales
    viento_umbral = df_ind["wspd"].quantile(0.90)
    lluvia_umbral = 5.0
    presion_umbral = df_ind["pres"].quantile(0.10)

    df_ind["viento_fuerte"] = (df_ind["wspd"] > viento_umbral).astype(int)
    df_ind["lluvia_intensa"] = (df_ind["prcp"] > lluvia_umbral).astype(int)
    df_ind["presion_baja"] = (df_ind["pres"] < presion_umbral).astype(int)

    return df_ind

### **Clasificación climática**

In [107]:
def clasificar_condicion(df):
    df = df.copy()
    
    score = (
        df["viento_fuerte"] +
        df["lluvia_intensa"] +
        df["presion_baja"])

    df["condicion_climatica"] = np.select(
        [score == 0, score == 1, score >= 2],
        ["Normal", "Condición moderada", "Condición severa"])
    
    return df

### **Ejecución principal + Clasificación**

In [109]:
if __name__ == "__main__":

    lat_lima = -12.0464
    lon_lima = -77.0428

    inicio = datetime(2024, 1, 1)
    fin = datetime(2024, 12, 31)

    df_clima = obtener_datos_climaticos(lat_lima, lon_lima, inicio, fin)
    df_limpio = limpiar_datos_climaticos(df_clima)
    df_indicadores = construir_indicadores(df_limpio)
    df_final = clasificar_condicion(df_indicadores)

    # Guardar CSV
    df_final.to_csv("senamhi_clima_indicadores.csv")

    # VISUALIZACIÓN EN PANDAS (como pediste)
    print("\n Vista previa del dataset final:")
    print(df_final.head())

    print("\n Resumen del dataset:")
    print(df_final.info())


 Vista previa del dataset final:
            tavg  tmin  tmax  prcp  wspd    pres  viento_fuerte  \
time                                                              
2024-01-01  23.4  22.0  25.2   1.0  19.1  1014.0              0   
2024-01-02  23.8  22.0  26.0   0.0  20.9  1013.4              1   
2024-01-03  23.5  22.0  25.3   0.0  21.5  1013.5              1   
2024-01-04  23.6  21.8  26.0   0.0  16.4  1014.1              0   
2024-01-05  23.9  22.0  26.0   0.0  24.7  1013.2              1   

            lluvia_intensa  presion_baja condicion_climatica  
time                                                          
2024-01-01               0             0              Normal  
2024-01-02               0             0  Condición moderada  
2024-01-03               0             0  Condición moderada  
2024-01-04               0             0              Normal  
2024-01-05               0             0  Condición moderada  

 Resumen del dataset:
<class 'pandas.core.frame.DataFr